In [1]:
import pymc as pm
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy import special
import arviz as az
import pandas as pd
from scipy import stats
import seaborn as sns
import pickle
import math
from collections import Counter

WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.


Could not locate executable g77
Could not locate executable f77
Could not locate executable ifort
Could not locate executable ifl
Could not locate executable f90
Could not locate executable DF
Could not locate executable efl
Could not locate executable gfortran
Could not locate executable f95
Could not locate executable g95
Could not locate executable efort
Could not locate executable efc
Could not locate executable flang
don't know how to compile Fortran code on platform 'nt'


WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
meta=pd.read_csv('meta_for_pymc.csv',index_col=0)

In [3]:
counts = pd.read_csv('geneName_raw_counts_from_deseq2.csv',index_col=0)

In [6]:
data=counts.loc[:,'CFLAR'].values

In [7]:
data.shape

(429,)

In [9]:
with pm.Model() as model:
    
    b0 = pm.Normal('Intercept', mu=0, sigma=10)
    b1 = pm.Normal('batch_pymc',mu=0, sigma=10)
    b2 = pm.Normal('log_nCount_RNA',mu=0, sigma=10)
    b3 = pm.Normal('log_nFeature_RNA',mu=0, sigma=10)
    
    theta = (b0 + b1*meta['batch_pymc'].values + b2*meta['log_nCount_RNA'].values + b3*meta['log_nFeature_RNA'].values)
    
    mu1 = pm.math.exp(theta)
    
    mu2_prior = pm.Uniform('mu2', lower=theta+1.2, upper=12)
    
    mu2 = pm.math.exp(mu2_prior)
    
    pois1 = pm.Poisson.dist(mu=mu1)  ## need to use '.dist' otherwise will get an error
    pois2 = pm.Poisson.dist(mu=mu2)
    
    zero_inflated = pm.DiracDelta.dist(0)

    w = pm.Dirichlet('w', a=np.array([1, 1 ,1]))

    like = pm.Mixture('mixture', w=w, comp_dists = [zero_inflated,pois1,pois2], observed=data)


In [ ]:
%%time
with model:
    try1_result = pm.sample(cores=2)